<a id='section_id0'></a>
<h1>Обучение моделей в Yandex DataSphere</h1>

1. [Фоновый запуск операций](#section_id1)
2. [Early Access Version: Фоновый запуск последовательности операций](#section_id2)
3. [Делегирование вычислений на кластера Data Proc](#section_id3) 
4. [Индикаторы загрузки памяти, CPU и GPU](#section_id4)  
5. [Поддержка TensorBoard](#section_id5)
6. [Узнать о Yandex DataSphere больше](#section_id6)

<a id='section_id1'></a>
## 1. Фоновый запуск операции
В Yandex DataSphere можно запустить операцию асинхронно. Вы можете запустить, например, процедуру оценки или обучение модели, и продолжить работу в ноутбуке. Выполнение операции будет происходить в фоне. Фоновые операции могут выполняться на прерываемых виртуальных машинах и ресурсах, поэтому могут выполняться дольше, чем обычные операции. Зато и тарифицируется исполнение таких операций  <a href="https://cloud.yandex.ru/docs/datasphere/pricing#async"> по другим правилам.</a> 
Чтобы выполнение операции происходило в фоновом режиме, в ячейке надо указать **#pragma async**. Основное ограничение фоновых ячеек - диск в них доступен только в read-only режиме.

In [ ]:
#Рассмотрим применение фоновой операции в обучении модели
#Установим обновленную версию TensorFlow. После этого шага не забудьте рестартовать kernel.
%pip install tensorflow -U

**Важно:** чтобы tensorflow > 2.4.x был совместим с CUDA нужно собрать докер-образ c новым драйвером. Этот сценарий описан в [Настройка рабочей среды в Yandex DataSphere](environment.ipynb#section_id0).

In [ ]:
#Импортируем нужные библиотеки
import tensorflow as tf
import datetime

In [ ]:
#Задаем модель
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

def create_model():
  return tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
  ])

In [ ]:
#Запускаем обучение модели в фоне
#pragma async
model = create_model()
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(x=x_train,
          y=y_train,
          epochs=5,
          validation_data=(x_test, y_test))
model = create_model()
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
#Пока в фоне происходит обучение модели, мы можем выполнять операции в основном потоке вычислений в ноутбуке
a = 5
print (a)

Чтобы прервать выполнение фоновой операции, нужно выделить ячейку, в которой она выполняется, и вызвать из пункта **Kernel** главного меню команду **Interrupt selected cell(s)**  
![](https://i.imgur.com/n6jHs54.png)

In [ ]:
#После окончания вычисления фоновой операции ее результат может понадобиться явно замержить
#Нам поможет команда %apply_state и идентификатор состояния, указанный в warning
%apply_state <checkpoint id>
#В результате увидим сообщение "State merged!"

<a id='section_id2'></a>
## 2. Early Access Version: Фоновый запуск последовательности операций
В режиме [Early Access Version](welcome_ru1.ipynb#section_id3) доступен фоновый запуск не только для одной операции, но и для последовательности операций. 
Фоновый запуск посдедовательности операций также может выполняться на прерываемых виртуальных машинах, и тарификация фонового запуска последовательности операций происходит <a href="https://cloud.yandex.ru/docs/datasphere/pricing#async"> по тем же правилам</a>, что и тарификация одной фоновой операции. 

В отличие от фонового запуска одной операции, для запуска последовательности нужно указать желаемое количество запусков, периодичность сохранения промежуточных результатов и правило выбора конечного результата. Для фонового запуска последовательных операций в ячейке надо указать команду **#pragma repeat**.

Запустим обучение модели, которую мы уже рассматривали [выше](#section_id2). В этом примере мы запускаем обучение модели 20 раз и сохраняем результат каждые 5 итераций. В состояние попадет модель, которая показала лучший результат на тестовых данных.

In [ ]:
#pragma repeat --iterations 20 --checkpoint-period 5 --max-by "model.evaluate(x_test, y_test, verbose=0)[1]"

model.fit(x_train, y_train, epochs=1, verbose=0)

In [ ]:
#Как и в случае однокраnной фоновой операции, замержим результат, используя идентификатор состояния (result id) нужной попытки
%apply_state <checkpoint id>

<a id='section_id3'></a>
## 3. Делегирование вычислений на кластера Data Proc 
Бесшовная интеграция DataSphere с сервисом Yandex Data Proc позволяет делегировать вычисления на кластерах Apache Spark™ прямо из ноутбука. Можно использовать уже существующий кластер, а можно создать его прямо из ноутбука DataSphere.
Чтобы вы могли создавать кластеры Data Proc из DataSphere или запускать уже существующие кластеры Data Proc, у проекта должны быть указаны:
- сервисный аккаунт, от имени которого будут производиться все операции с кластерами Data Proc.
- подсеть, в которой будет создаваться или из которой будет подключаться уже существующий кластер Data Proc. В рамках интеграции доступны только подсети, созданные в зоне доступности **ru-central1-a**.
Эти параметры можно выбрать в дополнительных настройках проекта в консоли.

![](https://i.imgur.com/VveEjRA.jpg)

Давайте создадим кластер прямо из ноутбука DataSphere: В разделе **File** главного меню выберем пункт **Data Proc Clusters**, по клику на него откроем окно создания кластера и зададим имя и размер нового кластера.
     
![](https://i.imgur.com/h057zc4.jpg)                     ![](https://i.imgur.com/SI31VES.png)

Чтобы запустить вычисление на DataProc, надо указать **#!spark --cluster <имя кластера>**. В примере ниже запустим вычисление на созданном нами кластере с именем **new**.

In [ ]:
#!spark --cluster new
import random

def inside(p):
    x, y = random.random(), random.random()
    return x*x + y*y < 1

NUM_SAMPLES = 1_000_000

count = sc.parallelize(range(0, NUM_SAMPLES)) \
             .filter(inside).count()
print("Pi is roughly %f" % (4.0 * count / NUM_SAMPLES))

<a id='section_id4'></a>
## 4. Индикаторы загрузки памяти, CPU и GPU
Теперь они видны в ноутбуке в верхнем правом углу

![](https://i.imgur.com/26S1lHz.jpeg)

<a id='section_id5'></a>
## 5. TensorBoard
DataSphere поддерживает TensorBoard. Давайте проиллюстрируем работу с ним на примере

In [ ]:
#Запустим еще раз обучение модели, описанной выше
model = create_model()
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model.fit(x=x_train, 
          y=y_train, 
          epochs=5, 
          validation_data=(x_test, y_test), 
          callbacks=[tensorboard_callback])

In [ ]:
#Теперь мы можем наблюдать прогресс на TensorBoard
%tensorboard --logdir logs/fit

Помимо отрисовки TensorBoard прямо в ноутбуке, можно воспользоваться панелью слева, где отображены все сессии TensorBoard, и перейти к графу TensorBoard в отдельной вкладке.   

![](https://i.imgur.com/XwmBC5l.jpeg)

<a id='section_id6'></a>
## 6. Узнать о Yandex DataSphere больше.
У нас есть довольно подробная <a href="https://cloud.yandex.ru/docs/datasphere/"> документация</a>. 

**Еще три полезных ноутбука**  
[Добро пожаловать в Yandex DataSphere](welcome_ru1.ipynb#section_id0)  
[О работе с данными и версионировании кода, переменных и состояния в Yandex DataSphere, а также о том, как поделиться результатами](dataflow.ipynb#section_id0)  
[Настройка рабочей среды в Yandex DataSphere](environment.ipynb#section_id0)

**Видео о DataSphere:**
- <a href="https://www.youtube.com/watch?v=udZcoKwb6aM&lc=UgyYeJGhB1WX-FvyqTJ4AaABAg"> вебинар "Новое в Yandex DataSphere для ML-разработки" (февраль 2021) </a>
- <a href="https://www.youtube.com/watch?v=_haQTawg940"> практический вебинар по DataSphere (ноябрь 2020) </a>
- <a href="https://youtu.be/1px5ZoN44cE"> доклад на Yandex.Scale 2020 о возможностях DataSphere (сентябрь 2020)</a>
- <a href="https://youtu.be/5y2-x9GcITs"> рассказ на Yandex.Scale 2020 о технической реализации DataSpher (сентябрь 2020)e</a>
- <a href="https://youtu.be/5o5OJOwDfaU"> практический вебинар, работа в DataSphere на примере публичных данных (август 2020)</a>
- <a href="https://www.youtube.com/watch?v=MtXpZ4RZAjk"> рассказ о DataSphere на конференции about:cloud  (май 2020) </a>
